# Development Notebook

In [1]:
EMBEDDINGS_DIRECTORY = './vstore' # directory of stored embeddings

In [2]:
from dotenv import load_dotenv
import os
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import json
from langchain.vectorstores import FAISS
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.schema import Document
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    PromptTemplate
)

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn at Miami University"
)

print(response.output_text)

Underneath the twinkling stars by the Miami University fountain, a playful unicorn with shimmering mane danced joyfully, sprinkling stardust that filled the air with dreams of adventure for all the students sleeping nearby.


In [ ]:
embedding_model = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")
vectorstore = FAISS.load_local(
  'scripts/vstore', 
  embeddings = embedding_model, 
  allow_dangerous_deserialization = True
  )
retriever = vectorstore.as_retriever(search_kwargs={"k": 10, "fetch_k": 20})

In [ ]:
retriever.get_relevant_documents("photos?") # Example query

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 27.12inputs/s]


[Document(id='4f6118a3-2199-4c1b-be77-056d97aca848', metadata={'id': '2019.13.38', 'title': None, 'creator': 'Felice Beato', 'date': '1860s', 'medium': 'Albumen photoprint', 'format': '9 1/4" x 11 1/2"', 'subject': 'Graphic Arts-Photos', 'donor': 'Miami University Art Museum Purchase', 'citation': 'Felice Beato, “[Untitled],”Richard and Carole Cocks Art Museum at Miami University, accessed May 23, 2025,https://miamiuniversityartmuseum.omeka.net/items/show/20613.', 'tags': [], 'collection_link': None}, page_content='Title: Unknown\nCreator: Felice Beato\nDate: 1860s\nMedium: Albumen photoprint\nFormat: 9 1/4" x 11 1/2"\nSubject: Graphic Arts-Photos\nDescription: Looking under an arch to another building, presumably a temple'),
 Document(id='f75546a4-c7c7-4250-abfb-e467744756bd', metadata={'id': '2019.13.36', 'title': 'The Children of a Brother of Mr. Guikado', 'creator': 'Felice Beato', 'date': '1860s', 'medium': 'Albumen photoprint', 'format': '9" x 11 3/4"', 'subject': 'Graphic Arts-P

In [ ]:
local_llm = OllamaLLM(model="llama3")
llm = ChatOllama(model="llama3", temperature=0)
llm_json_mode = ChatOllama(model="llama3", temperature=0, format="json")

In [ ]:
system_temp = (
    "You are an expert art historian working for the Richard and Carole Cocks Art Museum at Miami University. "
    "You will be provided with a question and a set of contextual documents related to artworks, artists, or cultural history.\n\n"
    "Your task is to analyze the documents carefully and provide a thoughtful, accurate, and well-supported response based strictly on the information provided.\n\n"
    "Question: {question}\n\n"
    "Context: {context}\n\n"
    "If the answer cannot be found in the provided context, respond with: 'I don't know.'"
)


In [ ]:
prompt = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["context", "question"],
                template=system_temp
            )
        )
    ]
)

In [ ]:
docs = retriever.get_relevant_documents("What is the medium of the artwork?")

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 24.97inputs/s]


In [ ]:
rag_chain = (
    RunnablePassthrough.assign(context=lambda x: "\n\n".join(d.page_content for d in retriever.get_relevant_documents(x["question"])))
    | prompt
    | llm
    | StrOutputParser()
)

# 4. Run the model (just provide the question!)
response = rag_chain.invoke({"question": "list the photographs and be sure to check if they are photos "})
print(response)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 17.78inputs/s]


What a fascinating collection of photographs! As an art historian, I'll do my best to provide a comprehensive answer based on the context provided.

**Photographs by Felice Beato:**

1. **Unknown**: A photograph showing a temple or shrine under an archway.
2. **The Children of a Brother of Mr. Guikado**: A portrait of two people looking across a body of water at buildings, likely in Japan.
3. **Unknown Temple**: A portrait of an unknown temple, possibly in Japan.
4. **Temples at Kamakura**: A photograph showing temples at Kamakura, Japan, with people walking around below.
5. **Temple of Asaxa Yedo**: A photograph of the Temple of Asaxa Yedo, likely located in Tokyo, Japan.
6. **The Temple of Choo-o-ng Kioto**: A portrait of a temple behind a group of trees, possibly located in Kyoto, Japan.

These photographs are all albumen prints from the 1860s, which is consistent with Beato's work during that period. He was a pioneering photographer who worked extensively in Asia, particularly in J

In [ ]:
response = rag_chain.invoke({"question": "Tell me more about the long voyage home "})
print(response)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 21.76inputs/s]


What a fascinating collection of artworks! Based on the provided context, I'll focus on "The Long Voyage Home" by Varujan Boghosian (1996).

"The Long Voyage Home" appears to be a mixed-media painting that depicts a wooden sailboat at night. The artwork features a paper sail, wood mast, painted background, and a circular wooden moon with glitter. This piece seems to evoke a sense of tranquility and serenity, perhaps capturing the peacefulness of being out on the water under the stars.

In comparison to other artworks in this collection, "The Long Voyage Home" stands out for its unique blend of materials and textures. The use of wood, paper, paint, and glue creates a visually interesting piece that invites the viewer to explore its various elements.

It's also worth noting that the title "The Long Voyage Home" might be interpreted as a metaphorical journey rather than just a literal one. This could lead to discussions about the themes of travel, exploration, or even the search for home 